In [2]:
!conda install -c conda-forge beautifulsoup4 --yes
import numpy as np
import pandas as pd
import requests
from pandas.io.json import json_normalize
import json
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    beautifulsoup4-4.8.1       |           py36_0         149 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.1-py36_0         conda-forge
    ca-certificates: 2019.8.28-0       --> 2019.9.11-hecc5488_0 conda-forge
    certifi:         2019.9.11-py36_0  --> 2019.9.11-py36_0     conda-forg

In [3]:
spURL= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(spURL).text
soup = BeautifulSoup(data, 'html.parser')

In [8]:
zipCode = []
borough = []
neighborhood = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        zipCode.append(cells[0].text)
        borough.append(cells[1].text)
        neighborhood.append(cells[2].text.rstrip('\n'))

In [13]:
df = pd.DataFrame({'PostalCode':zipCode,'Borough':borough,'Neighborhood':neighborhood})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [14]:
#Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']

In [17]:
#More than one neighborhood can exist in one postal code area, combine them
df= df.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))

In [51]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
for i in range(len(df['Neighborhood'].values)):
    if df.iloc[i,2] == 'Not assigned':
        df.iloc[i,2] = df.iloc[i,1]

In [54]:
df.shape

(103, 3)

In [55]:
df.to_csv('Toronto_DATA.csv')